<a href="https://colab.research.google.com/github/sbarreto10/data-science-2022/blob/main/SPOTIFY%20DATASET%20(TP3)/75_06_TP_3_KERASNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**LECTURA**

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import random
pd.options.mode.chained_assignment = None

from tqdm import tqdm

In [ ]:
trainDf = pd.read_parquet("/content/drive/MyDrive/tp3/train.parquet")
testDf = pd.read_parquet("/content/drive/MyDrive/tp3/test.parquet")

trainDf["genre"][trainDf[trainDf.genre == "Children's Music"].index[0]] = "Children’s Music"

#**MULTILABELING**

In [ ]:
def multi_label(df, fitting=[]):
    grp = df[["track_name","artist","genre"]]
    grp["1"] = 1
    grp["id"] = grp.apply(lambda x: (x.track_name, x.artist), axis=1)
    piv = grp.pivot_table(columns="genre", index="id", values="1", fill_value=0)
    for g in fitting:
        if g not in piv.columns:
            piv[g] = 0
    df["id"] = df.apply(lambda x: (x.track_name, x.artist), axis=1)
    df = df.drop_duplicates(subset=["id"], keep='first')

    return df.merge(piv,on="id"), piv.columns

trainDf, trainGenres = multi_label(trainDf)
testDf, genreList = multi_label(testDf, fitting=trainGenres)
genreList = genreList.tolist()

In [ ]:
trainDf, genreList = multi_label(trainDf)
testDf, genreList = multi_label(testDf, fitting=trainGenres)
genreList = genreList.tolist()

#**SPLIT**

In [ ]:
random.seed(3)
artistList = trainDf.artist.unique().tolist()
validationArtists = random.sample(artistList, int(0.2*len(artistList)))
trainDf, valDf = trainDf.query("artist not in @validationArtists"), trainDf.query("artist in @validationArtists")

#**PRE-KERAS PREPROCESSING**

In [ ]:
def preprocess(df):
    # DROPEO
    df = df.drop(columns = ["id","track_name","did","artist","a_genres","a_songs"])

    # IMPUTACIÓN DE NULOS
    s_labelMean = df["s-label"].mean()
    df.lyric = df.lyric.fillna("").astype(str)
    df.language = df.language.map(lambda x: "ot" if type(x)==type(None) else x)
    df["s-label"] = df["s-label"].fillna(s_labelMean)
    df["mode"] = df["mode"].map(lambda m: int(m=="Major"))
    df.lyric = df.apply(lambda x: "" if type(x.lyric)!=str else x.lyric, axis=1)

    return df

In [ ]:
trainDf = preprocess(trainDf)
valDf = preprocess(valDf)
testDf = preprocess(testDf)

In [ ]:
def fix_nan_lyrics(df):
    df.lyric = df.apply(lambda x: "" if type(x.lyric)!=str else x.lyric, axis=1)
    return df

trainDf = fix_nan_lyrics(trainDf)
valDf = fix_nan_lyrics(valDf)
testDf = fix_nan_lyrics(testDf)

#**RED NEURONAL**

In [ ]:
import keras
from keras import Imput
from keras.layers import Dense
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import TextVectorization
from keras.optimizers import SGD

In [ ]:
vectorize_layer = TextVectorization()(
    max_tokens=5000,
    output_mode="tf_idf",
    output_sequence_length=5))

In [ ]:
model = Sequential()

model.add(Input(shape=(1,), dtype=tf.string))

model.add(vectorize_layer)